Refer - https://blog.openmined.org/what-is-pygrid-demo/

<h2>Populate remote PyGrid nodes with labeled tensors </h2>
In this notebook, we will send data in a federated approach.

**NOTE:** At the time of running this notebook, we were running the grid components in background mode.  

Components:
 - PyGrid Network (http://localhost:5000)
 - PyGrid Node h1 (http://localhost:3000)
 - PyGrid Node h2 (http://localhost:3001)
 
Code implementation for this notebook has been referred from <a href="https://github.com/OpenMined/PySyft/blob/master/examples/tutorials/grid/federated_learning/mnist/Fed.Learning%20MNIST%20%5B%20Part-2%20%5D%20-%20Train%20a%20Model.ipynb">Fed.Learning MNIST [ Part-2 ] - Train a Model</a> tutorial

In [1]:
import syft as sy

#########<syft==0.2.8>#######################
# Dynamic FL -->
from syft.grid.clients.data_centric_fl_client import DataCentricFLClient

#Static FL -->
from syft.grid.clients.model_centric_fl_client import ModelCentricFLClient
#############################################

#########<syft==0.2.6>#######################
# # Dynamic FL -->
# from syft.grid.clients.dynamic_fl_client import DynamicFLClient

# #Static FL -->
# from syft.grid.clients.static_fl_client import StaticFLClient
#############################################

import torch as th
import pickle
import time
import torchvision
from torchvision import datasets, transforms
# import tqdm
# from ipywidgets import IntProgress

In [2]:
# import syft as sy
# import torch as th
# from syft.workers.node_client import NodeClient
# hook = sy.TorchHook(th)

In [3]:
hook = sy.TorchHook(th)
h1 = DataCentricFLClient(hook, "ws://0.0.0.0:3000")
h2 = DataCentricFLClient(hook, "ws://0.0.0.0:3001")
h1, h2

(<Federated Worker id:h1>, <Federated Worker id:h2>)

In [4]:
## HOSPITAL 1
data_description1 = """Description:
                        This dataset represents the birth records for the month of February.
                        The data is arranged in the following format:
                        
                        Columns:
                            Gender: 0-Male, 1-Female
                            Weight (Kilograms): Float value
                            Height (centimeters): Float value
                            
                        Shape: (5 X 3)"""

records1 = th.tensor(
[
    [1,3.5,47.3],
    [0,3.7,48.1],
    [0,3.9,50.0],
    [1,4.1,52.3],
    [0,4.1,49.7]
]                                                     
)

In [5]:
## HOSPITAL 2
data_description2 = """Description:
                        This dataset represents data for diabetes patients.
                        The data is arranged in the following format:
                        
                        Columns:
                            Gender: 0-Male, 1-Female
                            Weight (Kilograms): Float value
                            Height (centimeters): Float value
                            
                        Shape: (5 X 3)"""

records2 = th.tensor(
[
    [1,3.5,47.3],
    [0,3.7,48.1],
    [0,3.9,50.0]
]                                                     
)

In [6]:
# private_dataset = monthly_birth_records.private_tensor(allowed_users=("Bob", 
#                                                                     "Ana",
#                                                                     "Andy"))

In [7]:
private_dataset1 = records1.tag("#age-related",
                                     "#birth-records").describe(data_description1)

private_dataset2 = records2.tag("#age-related",
                                     "#diabetes").describe(data_description2)
private_dataset1, private_dataset2

(tensor([[ 1.0000,  3.5000, 47.3000],
         [ 0.0000,  3.7000, 48.1000],
         [ 0.0000,  3.9000, 50.0000],
         [ 1.0000,  4.1000, 52.3000],
         [ 0.0000,  4.1000, 49.7000]])
 	Tags: #birth-records #age-related 
 	Description: Description:...
 	Shape: torch.Size([5, 3]),
 tensor([[ 1.0000,  3.5000, 47.3000],
         [ 0.0000,  3.7000, 48.1000],
         [ 0.0000,  3.9000, 50.0000]])
 	Tags: #diabetes #age-related 
 	Description: Description:...
 	Shape: torch.Size([3, 3]))

In [8]:
data_pointer1 = private_dataset1.send(h1)
data_pointer2 = private_dataset2.send(h2)
data_pointer1, data_pointer2

((Wrapper)>[PointerTensor | me:55244868465 -> h1:24907776859]
 	Tags: #birth-records #age-related 
 	Shape: torch.Size([5, 3])
 	Description: Description:...,
 (Wrapper)>[PointerTensor | me:59808081628 -> h2:15340492678]
 	Tags: #diabetes #age-related 
 	Shape: torch.Size([3, 3])
 	Description: Description:...)

<h2>Disconnect nodes</h2>

In [9]:
h1.close()
h2.close()

### Go to the following address to search available tags:
http://0.0.0.0:5000/search-available-tags